In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt

### Week 5: Monet Painting Generation

#### Problem
This project involves analyzing various paintings by Monet and creating an ML model that can modify photos to have a similar aesthetic as Monet's paintings. This data is sourced from https://www.kaggle.com/competitions/gan-getting-started/overview, containing 300 paintings by Monet, as well as various photos to try and stylize.

#### EDA
Show a few visualizations like histograms. Describe any data cleaning procedures. Based on your EDA, what is your plan of analysis?


#### Model Architecture
Describe your model architecture and reasoning for why you believe that specific architecture would be suitable for this problem.

#### Results and Analysis
Run hyperparameter tuning, try different architectures for comparison, apply techniques to improve training or performance, and discuss what helped.

Includes results with tables and figures. There is an analysis of why or why not something worked well, troubleshooting, and a hyperparameter optimization procedure summary.

Copied from Week 4, might not apply.

#### Conclusion

Discuss and interpret results as well as learnings and takeaways. What did and did not help improve the performance of your models? What improvements could you try in the future?